## Limpieza de la data

Lo que haremos en esta fase:

1. Hacer una copia a la base de datos original (siempre hay hacer esto)
2. Ahora si procedemos hacer la limpieza:

    ✅ Datos faltantes (Nulos).
    
    ✅ Registros duplicados.
    
    ✅ Formatos inconsistentes (fechas, nombres, números).
    
    ✅ Valores atípicos (outliers).

In [7]:
# Cargar data
import pandas as pd
import sqlite3

# Creamos una variable y le asignamos la ruta del archivo
ruta = "../data/raw/ufo_data.db"
# Creamos la conexión
conexion = sqlite3.connect(ruta)
cursor = conexion.cursor()


In [8]:
# Crear una copia de la db
copia = "../data/processed/copia.db"

consulta = f"""
ATTACH DATABASE '{copia}' AS copia;
SELECT sql FROM sqlite_master WHERE type='table';
"""
cursor.executescript(consulta)

# Copiar las tablas a la base de datos de respaldo
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()

for tabla in tablas:
    nombre_tabla = tabla[0]
    consulta_copia = f"CREATE TABLE copia.{nombre_tabla} AS SELECT * FROM {nombre_tabla};"
    cursor.execute(consulta_copia)

print("✅ Copia de la BD creada correctamente...")


✅ Copia de la BD creada correctamente...


In [ ]:
# Cerrar conexion
# conexion.close()